In [476]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error as MSE, r2_score as r2
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [477]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [478]:
df=pd.read_csv('train.csv')

In [479]:
data=df.copy()

In [480]:
data = pd.get_dummies(data)

In [481]:
data.columns

Index(['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
       'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2',
       'Social_3', 'Healthcare_1', 'Helthcare_2', 'Shops_1', 'Price',
       'Ecology_2_A', 'Ecology_2_B', 'Ecology_3_A', 'Ecology_3_B', 'Shops_2_A',
       'Shops_2_B'],
      dtype='object')

In [482]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 23 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Price            10000 non-null float64
Ecology_2_A      10000 non-null uint8
Ecology_2_B      10000 non-null uint8
Ecology_3_A      10000 non-null uint8
Ecology_3_B      10000 non-null uint8
Shops_2_A        10000 non-null uint8
Shops_2_B        10000 non-null uint8
dtyp

In [483]:
data.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Healthcare_1,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,...,5202.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,...,1142.904460,1.319500,4.231300,214138.857399,0.009700,0.990300,0.027500,0.972500,0.082500,0.917500
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,...,1021.517264,1.493601,4.806341,92872.293865,0.098015,0.098015,0.163543,0.163543,0.275139,0.275139
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,...,0.000000,0.000000,0.000000,59174.778028,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,...,350.000000,0.000000,1.000000,153872.633942,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,...,900.000000,1.000000,3.000000,192269.644879,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,...,1548.000000,2.000000,6.000000,249135.462171,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,...,4849.000000,6.000000,23.000000,633233.466570,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [484]:
data=data.drop('Healthcare_1',axis=1)

In [485]:
data.loc[data.HouseYear==20052011,'HouseYear']=2011
data.loc[data.HouseYear==4968,'HouseYear']=1968

# Разбираемся с комнатами

In [486]:
data.loc[(data.Rooms>5)]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_3,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
377,5927,57,10.0,59.056975,36.223072,10.0,22,22.0,2002,0.090799,...,2,5,15,317265.323792,0,1,0,1,0,1
1454,8491,1,19.0,42.006046,21.779288,7.0,17,17.0,2014,0.007122,...,0,0,1,78364.616704,0,1,0,1,0,1
2170,14003,99,6.0,59.414334,38.702244,6.0,7,9.0,1969,0.033494,...,1,3,8,229661.964416,0,1,0,1,0,1
8849,14865,9,10.0,60.871266,38.420681,10.0,3,2.0,1994,0.161532,...,1,2,4,172329.270863,0,1,0,1,0,1


In [487]:
data.loc[1454,'Rooms']=1
data.loc[[377,2170,8849],'Rooms']=2

In [488]:
data.loc[(data.Rooms==0)]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_3,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
1397,12638,27,0.0,138.427694,136.215499,0.0,4,3.0,2016,0.075424,...,0,0,0,268394.744389,0,1,0,1,0,1
1981,7917,27,0.0,212.932361,211.231125,0.0,2,3.0,2008,0.211401,...,0,0,1,302211.260887,0,1,0,1,0,1
2269,7317,27,0.0,41.790881,NaN,0.0,13,0.0,1977,0.211401,...,0,0,1,98129.976788,0,1,0,1,0,1
3911,770,28,0.0,49.483501,NaN,0.0,16,0.0,2015,0.118537,...,1,1,0,217009.338463,0,1,0,1,0,1
4366,456,6,0.0,81.491446,NaN,0.0,4,0.0,1977,0.243205,...,0,0,0,212864.799112,0,1,0,1,0,1
4853,3224,27,0.0,2.377248,0.873147,0.0,1,0.0,1977,0.017647,...,0,0,0,126596.941798,0,1,0,1,0,1
6149,3159,88,0.0,38.697117,19.345131,9.0,9,16.0,1982,0.127376,...,3,3,9,158998.110646,0,1,0,1,0,1
8834,9443,27,0.0,87.762616,85.125471,0.0,5,15.0,1977,0.211401,...,0,0,1,219281.918007,0,1,0,1,0,1


In [489]:
data.loc[[1397,1981],'Rooms'] = 4
data.loc[[2269, 3911, 6149],'Rooms'] = 1
data.loc[[4366,8834],'Rooms'] = 3
data.drop(4853, inplace=True)


# Площадь

In [490]:
data.loc[data.Square>300]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_3,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
4262,28,9,2.0,604.705972,NaN,1.0,17,18.0,1977,0.161532,...,1,2,4,187717.242538,0,1,0,1,0,1
4690,2307,102,1.0,409.425181,410.639749,10.0,4,4.0,2016,0.238617,...,6,3,6,90470.430830,0,1,0,1,0,1
6977,11602,30,2.0,641.065193,638.163193,10.0,20,19.0,2019,0.000078,...,141,3,23,133529.681562,0,1,0,1,0,1


In [491]:
data.loc[(data.Square>300),'Square']=data['Square']/10
data.loc[[4262,4690,6977],'LifeSquare']=data['LifeSquare']/10
data.loc[[4262,4690,6977]]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_3,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
4262,28,9,2.0,60.470597,NaN,1.0,17,18.0,1977,0.161532,...,1,2,4,187717.242538,0,1,0,1,0,1
4690,2307,102,1.0,40.942518,41.063975,10.0,4,4.0,2016,0.238617,...,6,3,6,90470.430830,0,1,0,1,0,1
6977,11602,30,2.0,64.106519,63.816319,10.0,20,19.0,2019,0.000078,...,141,3,23,133529.681562,0,1,0,1,0,1


In [492]:
data_true = data.loc[(data.Square>(data.LifeSquare+data.KitchenSquare))&(data.Square>40)&(data.Square<90)&(data.LifeSquare>20)]
square_lifesquare=(data_true.Square/data_true.LifeSquare).mean()
data_kitchen = data.loc[(data.Square>(data.LifeSquare+data.KitchenSquare))&(data.Square>40)&(data.Square<90)&(data.LifeSquare>20)&(data.KitchenSquare>6)&(data.KitchenSquare<15)]
square_kitchen = (data_kitchen.Square/data_kitchen.KitchenSquare).mean()
print(square_lifesquare)
print(square_kitchen)

1.6136627508451011
6.8260281263003995


In [493]:
data.loc[data.LifeSquare>200]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_3,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
590,14990,23,2.0,48.449873,263.542020,5.0,6,5.0,1972,0.075779,...,3,0,2,141780.231857,0,1,0,1,0,1
1981,7917,27,4.0,212.932361,211.231125,0.0,2,3.0,2008,0.211401,...,0,0,1,302211.260887,0,1,0,1,0,1
1982,5548,86,5.0,275.645284,233.949309,26.0,12,37.0,2011,0.161976,...,5,3,7,455264.882666,0,1,1,0,0,1
4328,16550,27,3.0,81.694417,7480.592129,1.0,9,17.0,2016,0.017647,...,0,0,0,217357.492366,0,1,0,1,0,1
8437,15886,85,3.0,78.059331,461.463614,10.0,12,16.0,1998,0.037178,...,1,1,7,394253.299978,0,1,0,1,0,1
9910,16568,27,4.0,200.334539,201.627361,25.0,1,2.0,2013,0.041116,...,4,1,4,528560.506016,0,1,0,1,0,1


In [494]:
data.loc[[4328],'LifeSquare']=data.loc[[4328],'LifeSquare']/100
data.loc[[590,8437],'LifeSquare']=data.loc[[590,8437],'LifeSquare']/10

In [495]:
data.loc[(data.Square<20)]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_3,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
212,1748,88,2.0,5.497061,67.628717,1.0,24,22.0,1977,0.127376,...,3,3,9,412511.088764,0,1,0,1,0,1
1316,11526,27,1.0,4.633498,1.969969,1.0,18,1.0,1977,0.011654,...,0,0,0,107604.269441,0,1,0,1,0,1
1608,10202,6,1.0,2.596351,4.604943,1.0,3,25.0,2014,0.243205,...,0,0,0,137597.601458,0,1,0,1,0,1
1891,13685,0,1.0,17.498542,13.446544,0.0,7,5.0,1961,0.319809,...,16,5,8,88871.750230,0,1,0,1,0,1
3166,15127,150,1.0,17.046188,20.036232,8.0,7,12.0,1982,0.300323,...,6,1,9,74222.110465,0,1,0,1,0,1
3280,10527,27,1.0,4.380726,40.805837,1.0,10,17.0,2013,0.211401,...,0,0,1,97560.720383,0,1,0,1,0,1
3413,9487,5,1.0,5.129222,5.549458,1.0,1,1.0,1977,0.150818,...,4,4,5,369472.403061,0,1,0,1,0,1
3794,523,101,1.0,18.509177,15.170851,3.0,5,5.0,1970,0.225825,...,7,3,2,115845.861527,1,0,0,1,0,1
4739,12676,81,3.0,13.784865,15.988889,7.0,4,5.0,1960,0.319809,...,16,5,8,78388.806186,0,1,0,1,0,1
4900,4504,27,3.0,4.390331,5.610772,1.0,8,19.0,2016,0.211401,...,0,0,1,161379.067034,0,1,0,1,0,1


In [496]:
data.loc[[212,1316,1608,3280,3413,4900, 6392,8030,8283,9294,9613],'Square']=data.loc[[212,1316,1608,3280,3413,4900, 6392,8030,8283,9294,9613],'Square']*10
data.loc[[1316,1608,3413,4900, 6392,8283,9294,9613],'LifeSquare']=data.loc[[1316,1608,3413,4900, 6392,8283,9294,9613],'LifeSquare']*10

In [497]:
# поменяем местами площади, где Square < LifeSquare
data.loc[data.Square<data.LifeSquare,'Square']=data.loc[data.Square<data.LifeSquare,'LifeSquare']
data.head(5)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_3,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,...,5,0,11,184966.930730,0,1,0,1,0,1
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,...,1,1,16,300009.450063,0,1,0,1,0,1
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,...,0,1,3,220925.908524,0,1,0,1,0,1
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,...,3,0,5,175616.227217,0,1,0,1,0,1
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,...,1,2,4,150226.531644,0,1,0,1,0,1


In [498]:
#найдем варианты цен с подозрением на неадекватность
data.loc[(data.Price>300000)&(data.Square<30)]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_3,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
8283,15744,34,1.0,26.422186,26.422186,1.0,21,4.0,1977,0.069753,...,4,1,11,458378.777006,0,1,0,1,0,1


In [499]:
# удалим квартиры, где цены совсем неадекватна 
data.drop(8283, inplace=True)


In [500]:
data.loc[(data.Price<80000)&(data.Square>70)]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_3,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
696,7590,113,3.0,94.417148,56.749169,11.0,6,18.0,2008,0.050756,...,1,0,1,71057.273346,0,1,0,1,0,1
8373,14892,57,3.0,84.206197,49.920840,8.0,7,7.0,2005,0.090799,...,2,5,15,71319.376419,0,1,0,1,0,1
9046,15849,3,3.0,74.478225,47.539158,10.0,8,17.0,1984,0.050900,...,1,2,3,76652.041207,0,1,0,1,0,1


In [501]:
data.drop([696,8373,9046], inplace=True)

In [502]:
data['LifeSquare']=data['LifeSquare'].fillna(data['Square']/square_lifesquare)

# Кухня и жилая площадь - приведем в порядок

In [503]:
data['KitchenSquare']=data['KitchenSquare'].fillna(data['Square']/square_kitchen)

In [504]:
# Вместо значений площади кухни (>25 или <2) ставим значение в зависимости от площади квартиры
# Жилая площадь- Там где разница между площадями общей и жилой комнатами больше площади 
# кухни, устанавливаем жил.площадь из соотношения с общей


In [505]:
def LifeSq(df):
    df.loc[(df.KitchenSquare<2)|(df.KitchenSquare>24),'KitchenSquare']=df['Square']/square_kitchen
    df.loc[df.Square<(df.KitchenSquare+df.LifeSquare),'LifeSquare']=df['Square']/square_lifesquare
    df.loc[df.LifeSquare<(0.3*df.Square),'LifeSquare']=df['Square']/square_lifesquare
    return df

In [506]:
LifeSq(data).head(5)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_3,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
0,14038,35,2.0,47.981561,29.442751,6.000000,7,9.0,1969,0.089040,...,5,0,11,184966.930730,0,1,0,1,0,1
1,15053,41,3.0,65.683640,40.049543,8.000000,7,9.0,1978,0.000070,...,1,1,16,300009.450063,0,1,0,1,0,1
2,4765,53,2.0,44.947953,29.197612,6.584789,8,12.0,1968,0.049637,...,0,1,3,220925.908524,0,1,0,1,0,1
3,5809,58,2.0,53.352981,33.063279,9.000000,8,17.0,1977,0.437885,...,3,0,5,175616.227217,0,1,0,1,0,1
4,10783,99,1.0,39.649192,23.776169,7.000000,11,12.0,1976,0.012339,...,1,2,4,150226.531644,0,1,0,1,0,1


In [507]:
data.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_3,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
count,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000,...,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000
mean,8381.868734,50.402601,1.888344,56.308630,34.355920,8.258293,8.526663,12.611106,1984.863632,0.118889,...,8.042421,1.319360,4.230415,214165.539728,0.009705,0.990295,0.027514,0.972486,0.082541,0.917459
std,4858.385781,43.590431,0.811871,19.109648,12.584693,2.679873,5.240347,6.775358,18.413983,0.119045,...,23.837381,1.493385,4.805548,92827.048521,0.098039,0.098039,0.163583,0.163583,0.275201,0.275201
min,0.000000,0.000000,1.000000,15.988889,8.822988,2.000000,1.000000,0.000000,1910.000000,0.000000,...,0.000000,0.000000,0.000000,59174.778028,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4169.000000,20.000000,1.000000,41.883265,24.561071,6.000000,4.000000,9.000000,1974.000000,0.017647,...,0.000000,0.000000,1.000000,153915.286375,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
50%,8394.000000,36.000000,2.000000,52.623203,32.309585,8.000000,7.000000,13.000000,1977.000000,0.075424,...,2.000000,1.000000,3.000000,192302.751941,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
75%,12590.500000,75.000000,2.000000,65.981527,41.728528,10.000000,12.000000,17.000000,2001.000000,0.195781,...,5.000000,2.000000,6.000000,249148.484733,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
max,16798.000000,209.000000,5.000000,275.645284,233.949309,40.381504,42.000000,117.000000,2020.000000,0.521867,...,141.000000,6.000000,23.000000,633233.466570,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [508]:
data.loc[data.Square<(data.KitchenSquare+data.LifeSquare),'KitchenSquare']=data['Square']/square_kitchen


# Экология

In [509]:
# Заполняем нулевые значения медианными значениями параметра экологии из группировки по районам

In [510]:
def eco(df):
    df1=df.groupby(['DistrictId'])[['Ecology_1']].median()
    for i in df1.index:
        df.loc[(df.Ecology_1==0) & (df['DistrictId'] == i),'Ecology_1'] = df1.loc[i,'Ecology_1']
    return df

In [511]:
eco(data).head(5)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_3,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
0,14038,35,2.0,47.981561,29.442751,6.000000,7,9.0,1969,0.089040,...,5,0,11,184966.930730,0,1,0,1,0,1
1,15053,41,3.0,65.683640,40.049543,8.000000,7,9.0,1978,0.000070,...,1,1,16,300009.450063,0,1,0,1,0,1
2,4765,53,2.0,44.947953,29.197612,6.584789,8,12.0,1968,0.049637,...,0,1,3,220925.908524,0,1,0,1,0,1
3,5809,58,2.0,53.352981,33.063279,9.000000,8,17.0,1977,0.437885,...,3,0,5,175616.227217,0,1,0,1,0,1
4,10783,99,1.0,39.649192,23.776169,7.000000,11,12.0,1976,0.012339,...,1,2,4,150226.531644,0,1,0,1,0,1


# Приведем в порядок этажность

In [512]:
data.loc[data.Floor>data.HouseFloor,'HouseFloor']=data.loc[data.Floor>data.HouseFloor,'Floor']

In [513]:
data.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_3,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
count,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000,...,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000,9995.000000
mean,8381.868734,50.402601,1.888344,56.308630,34.355920,8.254196,8.526663,13.436018,1984.863632,0.120545,...,8.042421,1.319360,4.230415,214165.539728,0.009705,0.990295,0.027514,0.972486,0.082541,0.917459
std,4858.385781,43.590431,0.811871,19.109648,12.584693,2.677507,5.240347,6.093181,18.413983,0.118395,...,23.837381,1.493385,4.805548,92827.048521,0.098039,0.098039,0.163583,0.163583,0.275201,0.275201
min,0.000000,0.000000,1.000000,15.988889,8.822988,2.000000,1.000000,1.000000,1910.000000,0.000000,...,0.000000,0.000000,0.000000,59174.778028,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4169.000000,20.000000,1.000000,41.883265,24.561071,6.000000,4.000000,9.000000,1974.000000,0.020741,...,0.000000,0.000000,1.000000,153915.286375,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
50%,8394.000000,36.000000,2.000000,52.623203,32.309585,8.000000,7.000000,14.000000,1977.000000,0.075779,...,2.000000,1.000000,3.000000,192302.751941,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
75%,12590.500000,75.000000,2.000000,65.981527,41.728528,10.000000,12.000000,17.000000,2001.000000,0.195781,...,5.000000,2.000000,6.000000,249148.484733,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
max,16798.000000,209.000000,5.000000,275.645284,233.949309,40.381504,42.000000,117.000000,2020.000000,0.521867,...,141.000000,6.000000,23.000000,633233.466570,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [514]:
data.loc[data.HouseFloor>100]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_3,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
8854,78,30,2.0,66.811789,41.403812,9.787799,8,117.0,1977,0.000078,...,141,3,23,207248.37052,0,1,0,1,0,1


In [515]:
data.loc[data.HouseFloor>100,'HouseFloor']=data.loc[data.HouseFloor>100,'Floor']

In [516]:
data.loc[[6131,8599,8854]]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_3,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
6131,10806,5,1.0,51.944587,32.190485,7.609782,6,99.0,1977,0.150818,...,4,4,5,296127.115515,0,1,0,1,0,1
8599,9300,74,2.0,74.579809,46.217717,9.000000,5,99.0,1977,0.075779,...,3,0,2,243329.912579,0,1,0,1,0,1
8854,78,30,2.0,66.811789,41.403812,9.787799,8,8.0,1977,0.000078,...,141,3,23,207248.370520,0,1,0,1,0,1


# Обучаем модель

In [517]:
data.columns

Index(['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
       'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2',
       'Social_3', 'Helthcare_2', 'Shops_1', 'Price', 'Ecology_2_A',
       'Ecology_2_B', 'Ecology_3_A', 'Ecology_3_B', 'Shops_2_A', 'Shops_2_B'],
      dtype='object')

In [518]:
features = [
    'DistrictId', 
    'Rooms', 
    'Square', 
    'LifeSquare', 
    'KitchenSquare',
    'Floor', 
    'HouseFloor', 
    'HouseYear', 
    'Ecology_1', 
    'Social_1', 
    'Social_2',
    'Social_3', 
    'Helthcare_2', 
    'Shops_1', 
    'Ecology_2_A',
    'Ecology_2_B',
    'Ecology_3_A', 
    'Ecology_3_B', 
    'Shops_2_A', 
    'Shops_2_B'
]

In [519]:
X = data[features]
y = data['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [520]:
# Проведем нормирование данных

In [521]:
scaler = StandardScaler()

In [522]:
X_train_scaled = scaler.fit_transform(X_train)

In [523]:
X_test_scaled = scaler.transform(X_test)

In [524]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns = features)

In [525]:
X_test_scaled = pd.DataFrame(X_test_scaled, columns = features)

In [526]:
model = RF(n_estimators = 2000,  max_depth = 8, random_state = 42, n_jobs = 8)

In [527]:
%%time
model.fit(X_train_scaled, y_train)

Wall time: 10.6 s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=8,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=2000, n_jobs=8,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [528]:
pred_train = model.predict(X_train_scaled)

In [529]:
r2(y_train, pred_train)

0.7896310334806735

In [530]:
# Поработаем с параметрами модели

In [531]:
%%time
score_train = 0
score_test = 0
best_dep = 0
for i in range(100, 2000, 100):
        model = RF(n_estimators = i,  max_depth = 15, random_state = 42, n_jobs = 8)
        model.fit(X_train, y_train)
        pred_train = model.predict(X_train)
        r2_train = r2(y_train, pred_train)
        pred_test = model.predict(X_test)
        r2_test = r2(y_test, pred_test)
        if r2_test > score_test:
                score_test = r2_test
                score_train = r2_train
                best_dep = i
print(score_test, best_dep, score_train)

0.7371254767206823 1200 0.9401311545402454
Wall time: 4min 34s


In [532]:
model = RF(n_estimators = 1200,  max_depth = 15, random_state = 42, n_jobs = 8)

In [533]:
%%time
model.fit(X_train, y_train)

Wall time: 15.1 s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1200, n_jobs=8,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [534]:
r2_train = r2(y_train, pred_train)

In [535]:
r2_train

0.7896310334806735

In [536]:
pred_test = model.predict(X_test)

In [537]:
r2_test = r2(y_test, pred_test)

In [538]:
print(r2_test)

0.7371254767206823


# Test dataset

In [539]:
tes = pd.read_csv('test.csv')

In [540]:
test = tes.copy()

In [541]:
test = test.drop(['Healthcare_1'], axis = 1)

In [542]:
test = pd.get_dummies(test)

In [543]:
test.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_2,Social_3,Helthcare_2,Shops_1,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
count,5000.000000,5000.000000,5000.000000,5000.000000,3959.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,...,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,8412.595400,51.279200,1.910000,56.449500,36.158810,5.976800,8.632000,12.601000,1984.392600,0.119874,...,5406.900000,8.262600,1.31940,4.242800,0.009600,0.990400,0.029800,0.970200,0.082400,0.917600
std,4832.674037,44.179466,0.838594,19.092787,17.825287,9.950018,5.483228,6.789213,18.573149,0.120070,...,4026.614773,23.863762,1.47994,4.777365,0.097518,0.097518,0.170052,0.170052,0.275001,0.275001
min,1.000000,0.000000,0.000000,1.378543,0.333490,0.000000,1.000000,0.000000,1908.000000,0.000000,...,168.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4221.750000,21.000000,1.000000,41.906231,23.092026,1.000000,4.000000,9.000000,1973.000000,0.019509,...,1564.000000,0.000000,0.00000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
50%,8320.500000,37.000000,2.000000,52.921340,32.925087,6.000000,7.000000,12.000000,1977.000000,0.072158,...,5285.000000,2.000000,1.00000,3.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
75%,12598.250000,77.000000,2.000000,66.285129,45.174091,9.000000,12.000000,17.000000,2000.000000,0.195781,...,7287.000000,5.000000,2.00000,6.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
max,16795.000000,212.000000,17.000000,223.453689,303.071094,620.000000,78.000000,99.000000,2020.000000,0.521867,...,19083.000000,141.000000,6.00000,23.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [544]:
test.loc[test.Square<test.LifeSquare,'Square']=test.loc[test.Square<test.LifeSquare,'LifeSquare']
test.head(5)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_2,Social_3,Helthcare_2,Shops_1,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,...,2748,1,0,0,0,1,0,1,0,1
1,15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,...,1437,3,0,2,0,1,0,1,0,1
2,5480,190,1.0,15.948246,15.948246,12.0,2,5.0,1909,0.000000,...,7538,87,5,5,0,1,0,1,0,1
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,...,4583,3,3,3,0,1,0,1,0,1
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,...,629,1,0,0,0,1,0,1,1,0


In [545]:
test.loc[(test.Square<13)|(test.Square>150)]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_2,Social_3,Helthcare_2,Shops_1,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
66,9011,53,1.0,1.378543,1.353573,1.0,1,1.0,1977,0.049637,...,7759,0,1,3,0,1,0,1,0,1
410,11533,94,2.0,303.071094,303.071094,6.0,5,12.0,1974,0.521867,...,6149,0,0,0,0,1,0,1,0,1
608,16401,30,1.0,4.338755,4.338755,1.0,2,1.0,1977,0.000078,...,6398,141,3,23,0,1,0,1,0,1
837,2138,27,1.0,5.647458,1.501582,1.0,1,1.0,1977,0.017647,...,469,0,0,0,0,1,0,1,0,1
1165,10120,6,1.0,5.100672,3.861780,1.0,3,1.0,1977,0.243205,...,1564,0,0,0,0,1,0,1,0,1
1264,13589,27,3.0,173.974480,76.289475,9.0,3,3.0,2017,0.041116,...,14892,4,1,4,0,1,0,1,0,1
1499,7769,124,4.0,189.679576,114.708673,4.0,19,40.0,2013,0.174739,...,3379,9,0,3,0,1,0,1,0,1
2039,3357,27,3.0,170.476326,75.973612,8.0,2,2.0,2017,0.041116,...,14892,4,1,4,0,1,0,1,0,1
2557,16053,17,4.0,169.901701,169.901701,0.0,4,0.0,2013,0.093443,...,4635,5,2,4,0,1,0,1,0,1
3217,4058,27,6.0,223.453689,104.113552,16.0,2,2.0,2017,0.041116,...,14892,4,1,4,0,1,0,1,0,1


In [546]:
test.loc[(test.Square<13),['Square','LifeSquare']]=test.loc[(test.Square<13),['Square','LifeSquare']]*10


In [547]:
test.loc[test.LifeSquare>150]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_2,Social_3,Helthcare_2,Shops_1,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
410,11533,94,2.0,303.071094,303.071094,6.0,5,12.0,1974,0.521867,...,6149,0,0,0,0,1,0,1,0,1
2557,16053,17,4.0,169.901701,169.901701,0.0,4,0.0,2013,0.093443,...,4635,5,2,4,0,1,0,1,0,1


In [548]:
test['LifeSquare']=test['LifeSquare'].fillna(test['Square']/square_lifesquare)

In [549]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 21 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       5000 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Ecology_2_A      5000 non-null uint8
Ecology_2_B      5000 non-null uint8
Ecology_3_A      5000 non-null uint8
Ecology_3_B      5000 non-null uint8
Shops_2_A        5000 non-null uint8
Shops_2_B        5000 non-null uint8
dtypes: float64(6), int64(9), uint8(6)
memory usage: 615.3 KB


In [550]:
# Комнаты

In [551]:
test.loc[(test.Rooms>5)]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_2,Social_3,Helthcare_2,Shops_1,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
2071,10793,23,6.0,110.750226,68.632820,0.0,2,2.0,2015,0.014073,...,475,0,0,0,0,1,0,1,0,1
3217,4058,27,6.0,223.453689,104.113552,16.0,2,2.0,2017,0.041116,...,14892,4,1,4,0,1,0,1,0,1
3398,1435,111,17.0,52.866107,32.528342,8.0,15,17.0,1987,0.093443,...,4635,5,2,4,0,1,0,1,0,1


In [552]:
test.loc[[2071],'Rooms']=3
test.loc[[3217],'Rooms']=5
test.loc[[3398],'Rooms']=2

In [553]:
test.loc[(test.Rooms==0)]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_2,Social_3,Helthcare_2,Shops_1,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
2406,3343,58,0.0,116.824201,113.692424,0.0,3,3.0,1977,0.437885,...,5735,3,0,5,0,1,0,1,0,1
2524,10729,27,0.0,76.345154,42.820796,12.0,14,0.0,1977,0.017647,...,469,0,0,0,0,1,0,1,0,1


In [554]:
test.loc[[2406],'Rooms']=3
test.loc[[2524],'Rooms']=2

# Корректировка этажей

In [555]:
test.loc[test.Floor>test.HouseFloor,'HouseFloor']=test.loc[test.Floor>test.HouseFloor,'Floor']

In [556]:
test.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_2,Social_3,Helthcare_2,Shops_1,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,...,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,8412.595400,51.279200,1.907200,56.639292,36.463690,5.976800,8.632000,13.395000,1984.392600,0.119874,...,5406.900000,8.262600,1.31940,4.242800,0.009600,0.990400,0.029800,0.970200,0.082400,0.917600
std,4832.674037,44.179466,0.807415,19.326774,16.715499,9.950018,5.483228,6.231945,18.573149,0.120070,...,4026.614773,23.863762,1.47994,4.777365,0.097518,0.097518,0.170052,0.170052,0.275001,0.275001
min,1.000000,0.000000,1.000000,13.305531,0.333490,0.000000,1.000000,1.000000,1908.000000,0.000000,...,168.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4221.750000,21.000000,1.000000,42.023090,24.413782,1.000000,4.000000,9.000000,1973.000000,0.019509,...,1564.000000,0.000000,0.00000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
50%,8320.500000,37.000000,2.000000,52.957761,33.515870,6.000000,7.000000,14.000000,1977.000000,0.072158,...,5285.000000,2.000000,1.00000,3.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
75%,12598.250000,77.000000,2.000000,66.416189,44.992137,9.000000,12.000000,17.000000,2000.000000,0.195781,...,7287.000000,5.000000,2.00000,6.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
max,16795.000000,212.000000,5.000000,303.071094,303.071094,620.000000,78.000000,99.000000,2020.000000,0.521867,...,19083.000000,141.000000,6.00000,23.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [557]:
test.loc[test.HouseFloor>100,'HouseFloor']=test.loc[test.HouseFloor>100,'Floor']

In [558]:
# Жилая пл и кухня 

In [559]:
LifeSq(test).head(5)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_2,Social_3,Helthcare_2,Shops_1,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
0,725,58,2.0,49.882643,33.432782,6.000000,6,14.0,1972,0.310199,...,2748,1,0,0,0,1,0,1,0,1
1,15856,74,2.0,69.263183,42.922961,10.146923,6,6.0,1977,0.075779,...,1437,3,0,2,0,1,0,1,0,1
2,5480,190,1.0,15.948246,9.883259,12.000000,2,5.0,1909,0.000000,...,7538,87,5,5,0,1,0,1,0,1
3,15664,47,2.0,73.046609,51.940842,9.000000,22,22.0,2007,0.101872,...,4583,3,3,3,0,1,0,1,0,1
4,14275,27,1.0,47.527111,29.452939,6.962630,17,17.0,2017,0.072158,...,629,1,0,0,0,1,0,1,1,0


In [560]:
test.loc[test.Square<(test.KitchenSquare+test.LifeSquare)]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_2,Social_3,Helthcare_2,Shops_1,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
2,5480,190,1.0,15.948246,9.883259,12.0,2,5.0,1909,0.0,...,7538,87,5,5,0,1,0,1,0,1


In [561]:
test.loc[[2]]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_2,Social_3,Helthcare_2,Shops_1,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
2,5480,190,1.0,15.948246,9.883259,12.0,2,5.0,1909,0.0,...,7538,87,5,5,0,1,0,1,0,1


In [562]:
test.loc[test.Square<(test.KitchenSquare+test.LifeSquare),'KitchenSquare']=test['Square']/square_kitchen

# Экология

In [563]:
eco(test).head(5)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_2,Social_3,Helthcare_2,Shops_1,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
0,725,58,2.0,49.882643,33.432782,6.000000,6,14.0,1972,0.310199,...,2748,1,0,0,0,1,0,1,0,1
1,15856,74,2.0,69.263183,42.922961,10.146923,6,6.0,1977,0.075779,...,1437,3,0,2,0,1,0,1,0,1
2,5480,190,1.0,15.948246,9.883259,2.336387,2,5.0,1909,0.000000,...,7538,87,5,5,0,1,0,1,0,1
3,15664,47,2.0,73.046609,51.940842,9.000000,22,22.0,2007,0.101872,...,4583,3,3,3,0,1,0,1,0,1
4,14275,27,1.0,47.527111,29.452939,6.962630,17,17.0,2017,0.072158,...,629,1,0,0,0,1,0,1,1,0


In [564]:
price_pred = model.predict(test[features])

In [565]:
test['Price'] = price_pred

In [566]:
test[['Id', 'Price']].to_csv('DKhoshafyan_prediction.csv', index = None)